In [ ]:
import os
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/NLP/zs1

/content/drive/.shortcut-targets-by-id/1x6VDl8Zpmr-99phU-BoEaMo0ZKhEnC38/zs1


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Activation,Dropout,LSTM
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras import regularizers
import numpy as np
import pandas as pd
import tqdm

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
#getting data
df = pd.read_csv('/content/drive/MyDrive/NLP/zs1/train_file.csv')

In [ ]:
text1 = df['Title'].tolist()
text2 = df['Headline'] .tolist()

In [ ]:
y1 = df['SentimentTitle'] 
y2 = df['SentimentHeadline']

In [ ]:
token = Tokenizer()
token.fit_on_texts(text1)

In [ ]:
token2 = Tokenizer()
token2.fit_on_texts(text2)

In [ ]:
vocab1_size = len(token.index_word) + 1
vocab1_size

29335

In [ ]:
vocab2_size = len(token2.index_word) + 1
vocab2_size

42684

In [ ]:
#converting to sequences
encoded_text1 = token.texts_to_sequences(text1)

In [ ]:
#converting to sequences
encoded_text2 = token2.texts_to_sequences(text2)

In [ ]:
encoded_text1

In [ ]:
encoded_text2

In [ ]:
#taking max_length as 120
max_length = 120
X1 = pad_sequences(encoded_text1, maxlen=max_length,padding='post')

In [ ]:
X2 = pad_sequences(encoded_text2, maxlen=max_length,padding='post')

In [ ]:
print(X1)

[[    3  1877 16178 ...     0     0     0]
 [   11   326    20 ...     0     0     0]
 [ 8281  7284    27 ...     0     0     0]
 ...
 [  546   589  4616 ...     0     0     0]
 [    5   467   378 ...     0     0     0]
 [ 3759     8 13252 ...     0     0     0]]


In [ ]:
print(X2)

[[    9  5498 10338 ...     0     0     0]
 [ 2461 24612   336 ...     0     0     0]
 [18746 11163 15692 ...     0     0     0]
 ...
 [  104   276   534 ...     0     0     0]
 [  643   106   613 ...     0     0     0]
 [   18    74    85 ...     0     0     0]]


In [ ]:
X1.shape

(55932, 120)

In [ ]:
X2.shape

(55932, 120)

In [ ]:
X_train1 , X_test1 , y_train1 , y_test1 = train_test_split(X1,y1,random_state=42,test_size=0.2)

In [ ]:
X_train2 , X_test2 , y_train2 , y_test2 = train_test_split(X2,y2,random_state=42,test_size=0.2)

In [ ]:
#building model
vec_size1 = 300 #300 dimensions

model1 = Sequential()
model1.add(Embedding(vocab1_size,vec_size1,input_length=max_length))
model1.add(Conv1D(64,8,activation='tanh'))
model1.add(MaxPooling1D(2))
model1.add(Dropout(0.5))
model1.add(Dense(32,activation='tanh'))
model1.add(Dropout(0.5))
model1.add(Dense(16,activation='tanh'))
model1.add(GlobalMaxPooling1D())
model1.add(Dense(1))

model1.compile(optimizer = 'adam', loss='mean_absolute_error')


In [ ]:
#building model
vec_size2 = 300 #300 dimensions

model2 = Sequential()
model2.add(Embedding(vocab2_size,vec_size2,input_length=max_length))
model2.add(Conv1D(64,8,activation='tanh'))
model2.add(MaxPooling1D(2))
model2.add(Dropout(0.5))
model2.add(Dense(32,activation='tanh'))
model2.add(Dropout(0.5))
model2.add(Dense(16,activation='tanh'))
model2.add(GlobalMaxPooling1D())
model2.add(Dense(1))

model2.compile(optimizer = 'adam', loss='mean_absolute_error')


In [ ]:
#model for lstm title
#building a better model
#building model
vec_size11 = 300 #300 dimensions

model11 = Sequential()
model11.add(Embedding(vocab1_size,vec_size11,input_length=max_length))
model11.add(Conv1D(64,8,activation='tanh'))
model11.add(MaxPooling1D(2))
model11.add(LSTM(64,activation='tanh'))
model11.add(Dropout(0.5))
model11.add(Dense(32,activation='tanh'))
model11.add(Dropout(0.5))
model11.add(Dense(16,activation='tanh'))
model11.add(Dense(8,activation='tanh'))
model11.add(Dense(1))

model11.compile(optimizer = 'adam', loss='mean_absolute_error')

In [ ]:
model11.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 120, 300)          8800500   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 113, 64)           153664    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 56, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)               

In [ ]:
#adding callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint('dl1_model1.h5',save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss' , patience=10 , restore_best_weights=True)

In [ ]:

#fitting model
%%time
history1 = model1.fit(X_train1,y_train1,epochs=100,validation_data=(X_test1,y_test1),callbacks=[checkpoint,earlystopper], verbose=2)

Epoch 1/100
1399/1399 - 24s - loss: 0.0877 - val_loss: 0.1600
Epoch 2/100
1399/1399 - 21s - loss: 0.0670 - val_loss: 0.1608
Epoch 3/100
1399/1399 - 21s - loss: 0.0613 - val_loss: 0.0943
Epoch 4/100
1399/1399 - 21s - loss: 0.0585 - val_loss: 0.0994
Epoch 5/100
1399/1399 - 21s - loss: 0.0560 - val_loss: 0.0841
Epoch 6/100
1399/1399 - 21s - loss: 0.0539 - val_loss: 0.0614
Epoch 7/100
1399/1399 - 21s - loss: 0.0523 - val_loss: 0.0682
Epoch 8/100
1399/1399 - 21s - loss: 0.0511 - val_loss: 0.0603
Epoch 9/100
1399/1399 - 21s - loss: 0.0497 - val_loss: 0.0677
Epoch 10/100
1399/1399 - 21s - loss: 0.0491 - val_loss: 0.0619
Epoch 11/100
1399/1399 - 21s - loss: 0.0480 - val_loss: 0.0589
Epoch 12/100
1399/1399 - 21s - loss: 0.0474 - val_loss: 0.0590
Epoch 13/100
1399/1399 - 21s - loss: 0.0468 - val_loss: 0.0603
Epoch 14/100
1399/1399 - 21s - loss: 0.0468 - val_loss: 0.0614
Epoch 15/100
1399/1399 - 21s - loss: 0.0467 - val_loss: 0.0623
Epoch 16/100
1399/1399 - 21s - loss: 0.0460 - val_loss: 0.0599
E

In [ ]:
checkpoint2 = ModelCheckpoint('dl1_model2.h5',save_best_only=True)
earlystopper2 = EarlyStopping(monitor='val_loss' , patience=10 , restore_best_weights=True)

In [ ]:
#lstm title
checkpoint11 = ModelCheckpoint('dl2_model1.h5',save_best_only=True)
earlystopper11 = EarlyStopping(monitor='val_loss' , patience=10 , restore_best_weights=True)

In [ ]:

#fitting model
%%time
history2 = model2.fit(X_train2,y_train2,epochs=100,validation_data=(X_test2,y_test2),callbacks=[checkpoint2,earlystopper2])

In [ ]:
#training for lstm title
#fitting model 
%%time
history11 = model11.fit(X_train1,y_train1,epochs=100,validation_data=(X_test1,y_test1),callbacks=[checkpoint11,earlystopper11])

Epoch 1/100
1399/1399 [==============================] - 40s 27ms/step - loss: 0.0997 - val_loss: 0.0967
Epoch 2/100
1399/1399 [==============================] - 37s 26ms/step - loss: 0.0971 - val_loss: 0.0962
Epoch 3/100
1399/1399 [==============================] - 37s 26ms/step - loss: 0.0970 - val_loss: 0.0974
Epoch 4/100
1399/1399 [==============================] - 37s 26ms/step - loss: 0.0970 - val_loss: 0.0963
Epoch 5/100
1399/1399 [==============================] - 35s 25ms/step - loss: 0.0970 - val_loss: 0.0969
Epoch 6/100
1399/1399 [==============================] - 35s 25ms/step - loss: 0.0969 - val_loss: 0.0965
Epoch 7/100
1399/1399 [==============================] - 35s 25ms/step - loss: 0.0969 - val_loss: 0.0964
Epoch 8/100
1399/1399 [==============================] - 35s 25ms/step - loss: 0.0968 - val_loss: 0.0964
Epoch 9/100
1399/1399 [==============================] - 35s 25ms/step - loss: 0.0968 - val_loss: 0.0963
Epoch 10/100
1399/1399 [==============================]

In [ ]:
#lstm model for headline
#building a better model
#building model
vec_size21 = 300 #300 dimensions

model21 = Sequential()
model21.add(Embedding(vocab2_size,vec_size21,input_length=max_length))
model21.add(Conv1D(64,8,activation='tanh'))
model21.add(MaxPooling1D(2))
model21.add(LSTM(64,activation='tanh'))
model21.add(Dropout(0.5))
model21.add(Dense(32,activation='tanh'))
model21.add(Dropout(0.5))
model21.add(Dense(16,activation='tanh'))
model21.add(Dense(8,activation='tanh'))
model21.add(Dense(1))

model21.compile(optimizer = 'adam', loss='mean_absolute_error')

In [ ]:
model21.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 120, 300)          12805200  
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 113, 64)           153664    
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 56, 64)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_11 (Dropout)         (None, 32)               

In [ ]:
#lstm head
checkpoint21 = ModelCheckpoint('dl2_model2.h5',save_best_only=True)
earlystopper21 = EarlyStopping(monitor='val_loss' , patience=10 , restore_best_weights=True)

In [ ]:
#training for lstm title
#fitting model 
%%time
history21 = model21.fit(X_train2,y_train2,epochs=100,validation_data=(X_test2,y_test2),callbacks=[checkpoint21,earlystopper21])

Epoch 1/100
1399/1399 [==============================] - 43s 29ms/step - loss: 0.1132 - val_loss: 0.1099
Epoch 2/100
1399/1399 [==============================] - 40s 29ms/step - loss: 0.1107 - val_loss: 0.1099
Epoch 3/100
1399/1399 [==============================] - 40s 29ms/step - loss: 0.1106 - val_loss: 0.1096
Epoch 4/100
1399/1399 [==============================] - 40s 29ms/step - loss: 0.1106 - val_loss: 0.1095
Epoch 5/100
1399/1399 [==============================] - 40s 29ms/step - loss: 0.1106 - val_loss: 0.1093
Epoch 6/100
1399/1399 [==============================] - 40s 29ms/step - loss: 0.1105 - val_loss: 0.1093
Epoch 7/100
1399/1399 [==============================] - 40s 29ms/step - loss: 0.1105 - val_loss: 0.1093
Epoch 8/100
1399/1399 [==============================] - 40s 29ms/step - loss: 0.1105 - val_loss: 0.1093
Epoch 9/100
1399/1399 [==============================] - 40s 29ms/step - loss: 0.1105 - val_loss: 0.1099
Epoch 10/100
1399/1399 [==============================]

In [ ]:
history1.history

In [ ]:
#doing the required preprocessing for test data
test = pd.read_csv('/content/drive/MyDrive/NLP/zs1/test_file.csv')

In [ ]:
test.head()

In [ ]:
test_tex1 = test['Title']
test_text2 = test['Headline']

In [ ]:
#preprocessing the test texts
token = Tokenizer()
token.fit_on_texts(test_tex1)
encoded_test_text1 = token.texts_to_sequences(test_tex1)
to_test1 = pad_sequences(encoded_test_text1, maxlen=max_length,padding='post')

In [ ]:
#preprocessing the test textst
token22 = Tokenizer()
token22.fit_on_texts(test_text2)
encoded_test_text2 = token.texts_to_sequences(test_text2)
to_test2 = pad_sequences(encoded_test_text2, maxlen=max_length,padding='post')

In [ ]:
#making prediction -- prediction for lstm model
prediction11 = model11.predict(to_test1)
prediction11

array([[-2.2472705e-07],
       [-2.2470886e-07],
       [-2.2471613e-07],
       ...,
       [-2.2472341e-07],
       [-2.2471249e-07],
       [-2.2473978e-07]], dtype=float32)

In [ ]:
#plotting actual vs predicted values
plt.scatter(range(20),prediction11,c='r')
plt.scatter(range(20),)

In [ ]:
#making prediction
prediction2 = model2.predict(to_test2)

In [ ]:
#predictions for lstm headline
prediction21 = model21.predict(to_test2)

In [ ]:
prediction21

array([[-0.02500014],
       [-0.02500014],
       [-0.02500008],
       ...,
       [-0.02500014],
       [-0.02500014],
       [-0.02500014]], dtype=float32)

In [ ]:
prediction11 = prediction11.tolist() #lstm

In [ ]:
prediction2 = prediction2.tolist()

In [ ]:
prediction21 = prediction21.tolist()

In [ ]:
#making a dataframe from the predictions
ids = test['IDLink'].tolist()

In [ ]:
submission_file_dl1.head()

,IDLink,SentimentTitle,SentimentHeadline
0,tFrqIR6Chj,[0.06598712503910065],[-0.160150408744812]
1,DVAaGErjlF,[-0.08446292579174042],[-0.09125955402851105]
2,OT9UIZm5M2,[0.05555082485079765],[0.1948482245206833]
3,lflGp3q2Fj,[-0.16036950051784515],[0.17200297117233276]
4,zDYG0SoovZ,[-0.040119338780641556],[-0.16512718796730042]


In [ ]:
import numpy as np
submission_file_dl1 = pd.DataFrame(np.column_stack([ids, prediction1, prediction2]), 
                               columns=['IDLink', 'SentimentTitle', 'SentimentHeadline'])

In [ ]:
import numpy as np
submission_file_dl2 = pd.DataFrame(np.column_stack([ids, prediction11, prediction21]), 
                               columns=['IDLink', 'SentimentTitle', 'SentimentHeadline'])

In [ ]:
submission_file_dl2.head()

,IDLink,SentimentTitle,SentimentHeadline
0,tFrqIR6Chj,-2.2472704586107284e-07,-0.025000140070915222
1,DVAaGErjlF,-2.2470885596703738e-07,-0.025000140070915222
2,OT9UIZm5M2,-2.2471613192465156e-07,-0.025000080466270447
3,lflGp3q2Fj,-2.2470885596703738e-07,-0.025000136345624924
4,zDYG0SoovZ,-2.2471249394584447e-07,-0.025000140070915222


In [ ]:
submission_file_dl2.to_csv('submission_file_dl2.csv',index=False)

#### Training model using  pretrained word vectors / embeddings

In [ ]:
#getting glove embeddings


In [ ]:
SEQUENCE_LENGTH = 120 # the length of all sequences (number of words per sample)
EMBEDDING_SIZE = 100  # Using 100-Dimensional GloVe embedding vectors
TEST_SIZE = 0.2

In [ ]:
def get_embedding_vectors(tokenizer, dim=100):
    embedding_index = {}
    with open(f"glove.6B.{dim}d.txt", encoding='utf8') as f:
        for line in tqdm.tqdm(f, "Reading GloVe"):
            values = line.split()
            word = values[0]
            vectors = np.asarray(values[1:], dtype='float32')
            embedding_index[word] = vectors

    word_index = tokenizer.word_index
    embedding_matrix = np.zeros((len(word_index)+1, dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            # words not found will be 0s
            embedding_matrix[i] = embedding_vector
            
    return embedding_matrix

In [ ]:
#creating the model
def get_model(tokenizer, lstm_units):
    """
    Constructs the model,
    Embedding vectors => LSTM => 2 output Fully-Connected neurons with softmax activation
    """
    # get the GloVe embedding vectors
    embedding_matrix = get_embedding_vectors(tokenizer)
    model_pt = Sequential()
    model_pt.add(Embedding(len(tokenizer.word_index)+1,
              EMBEDDING_SIZE,
              weights=[embedding_matrix],
              trainable=False,
              input_length=SEQUENCE_LENGTH))

    model_pt.add(LSTM(lstm_units))
    model_pt.add(Dropout(0.3))
    model_pt.add(Dense(10, activation="relu"))
    model_pt.add(Dense(1))
    # compile as rmsprop optimizer
    # aswell as with recall metric
    model_pt.compile(optimizer = 'adam', loss='mean_absolute_error')
    model_pt.summary()
    return model_pt

In [ ]:
model_pt1 = get_model(tokenizer=token,lstm_units = 64)

Reading GloVe: 400000it [00:15, 25538.00it/s]


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 100)          2933500   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                42240     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 10)                650       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 2,976,401
Trainable params: 42,901
Non-trainable params: 2,933,500
_________________________________________________________________


In [ ]:
#pretrained embeddings
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint31 = ModelCheckpoint('dl1_model3_embeddings.h5',save_best_only=True)
earlystopper31 = EarlyStopping(monitor='val_loss' , patience=10 , restore_best_weights=True)
# train the model
history31 = model_pt1.fit(X_train1,y_train1,epochs=100,validation_data=(X_test1,y_test1),callbacks=[checkpoint31,earlystopper31])

Epoch 1/100
1399/1399 [==============================] - 45s 27ms/step - loss: 0.0968 - val_loss: 0.0962
Epoch 2/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.0968 - val_loss: 0.0962
Epoch 3/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.0968 - val_loss: 0.0962
Epoch 4/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.0968 - val_loss: 0.0962
Epoch 5/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.0968 - val_loss: 0.0964
Epoch 6/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.0968 - val_loss: 0.0962
Epoch 7/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.0968 - val_loss: 0.0963
Epoch 8/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.0968 - val_loss: 0.0962
Epoch 9/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.0968 - val_loss: 0.0962
Epoch 10/100
1399/1399 [==============================]

In [ ]:
#fitting on headline input
checkpoint32 = ModelCheckpoint('dl2_model3_embeddings.h5',save_best_only=True)
earlystopper32 = EarlyStopping(monitor='val_loss' , patience=10 , restore_best_weights=True)
# train the model
history31 = model_pt1.fit(X_train2,y_train2,epochs=100,validation_data=(X_test2,y_test2),callbacks=[checkpoint32,earlystopper32])

Epoch 1/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.1104 - val_loss: 0.1093
Epoch 2/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.1103 - val_loss: 0.1093
Epoch 3/100
1399/1399 [==============================] - 37s 27ms/step - loss: 0.1103 - val_loss: 0.1093
Epoch 4/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.1103 - val_loss: 0.1095
Epoch 5/100
1399/1399 [==============================] - 37s 27ms/step - loss: 0.1104 - val_loss: 0.1093
Epoch 6/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.1104 - val_loss: 0.1093
Epoch 7/100
1399/1399 [==============================] - 37s 27ms/step - loss: 0.1103 - val_loss: 0.1093
Epoch 8/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.1103 - val_loss: 0.1093
Epoch 9/100
1399/1399 [==============================] - 38s 27ms/step - loss: 0.1103 - val_loss: 0.1093
Epoch 10/100
1399/1399 [==============================]

In [ ]:
#doing the required preprocessing for test data
test = pd.read_csv('/content/drive/MyDrive/NLP/zs1/test_file.csv')

In [ ]:
test_tex1 = test['Title']
test_text2 = test['Headline']

In [ ]:
#preprocessing the test texts
#preprocessing the test texts
token = Tokenizer()
token.fit_on_texts(test_tex1)
encoded_test_text1 = token.texts_to_sequences(test_tex1)
to_test1 = pad_sequences(encoded_test_text1, maxlen=max_length,padding='post')

In [ ]:
#preprocessing the test textst
token2 = Tokenizer()
token2.fit_on_texts(test_text2)
encoded_test_text2 = token.texts_to_sequences(test_text2)
to_test2 = pad_sequences(encoded_test_text2, maxlen=max_length,padding='post')

In [ ]:
#making prediction -- prediction for lstm model
prediction1 = model_pt1.predict(to_test1)
prediction1 = prediction1.tolist()

In [ ]:
#making prediction -- prediction for lstm model
prediction2 = model_pt1.predict(to_test2)
prediction2 = prediction2.tolist()

In [ ]:
#making a dataframe from the predictions
ids = test['IDLink'].tolist()

In [ ]:
import numpy as np
submission_file_dl4 = pd.DataFrame(np.column_stack([ids, prediction1, prediction2]), 
                               columns=['IDLink', 'SentimentTitle', 'SentimentHeadline'])

In [ ]:
submission_file_dl4.to_csv('submission_file_dl4.csv',index=False)